In [1]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
iML1515_fn = '../models/e_coli/iML1515.xml'
model = cobra.io.read_sbml_model(iML1515_fn)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [93]:
# Get transport reactions
transport_reactions = []
transported_mets = []
met_to_transport = {}
for r in model.reactions:
    if len(r.compartments) >1:
        if r.id[:7] == 'BIOMASS':
            continue
        
        transport_reactions.append(r)
        t_mets = [m.id[:-2] for m in r.metabolites.keys() if m.id[-2:] in ['_e', '_p']]
        for m_id in t_mets:
            try:
                t_list = met_to_transport[m_id]
            except KeyError:
                met_to_transport[m_id] = [r.id]
            else:
                met_to_transport[m_id] = t_list+[r.id]

In [94]:
met_to_transport = {key: list(set(lst)) for key, lst in met_to_transport.items()}

In [170]:
co_mets = ['h_p', 'h_c', 'pep_c', 'pyr_c', 'pi_c', 'adp_c', 'atp_c', 'h2o_c']
all_data = []
for k, (key, lst) in enumerate(met_to_transport.items()):
    if key == 'h':
        continue
    for r_id in lst:
        r = model.reactions.get_by_id(r_id)
        # print(r.id, r.reaction, r.bounds)
        # print(r.compartments)
        # Remove reactions that occurs across the periplasmic membrane, but are not transport reactions per se
        mets = [m.id for m in r.metabolites]
        ttype = 'None'
        for pair in [['q8_c','q8h2_c'],['2dmmql8_c', '2dmmq8_c'], ['mql8_c','mqn8_c']]:
            if len(set(pair+mets)) == len(mets):
                # print('Skipping {0}'.format(r_id))
                ttype = 'Not transport'
                continue
                
        if 'murein5p5p_p' in mets:
            ttype = 'Not transport'
        elif 'dsbdox_c' in mets:
            ttype = 'Not transport'
        elif 'dsbcox_c' in mets:
            ttype = 'Not transport'
            
        if ttype == 'Not transport':
            continue
        
        
        # Check compartments involved
        membrane = '???'
        if r.compartments == {'c', 'p'}:
            membrane = 'inner'
        elif r.compartments == {'e', 'p'}:
            membrane = 'outer'
        elif len(r.compartments) == 3:
            # Remove co mets
            compartments = set([x.rsplit('_')[-1] for x in mets if not x in co_mets])
            if compartments == {'c', 'p'}:
                membrane = 'inner'
            elif compartments == {'e', 'p'}:
                membrane = 'outer'

        if len(mets)==2:
            ttype = 'Diffusion'
        elif 'atp_c' in mets:
            ttype = 'ATP-binding cassette transporter'
        elif 'pep_c' in mets:
            ttype = 'PEP group translocation'
        elif 'h_p' in mets:
            s_h = r.metabolites[model.metabolites.get_by_id('h_p')]
            s_m = r.metabolites[model.metabolites.get_by_id('{0}_p'.format(key))]
            if s_h*s_m == 1:
                ttype = 'Proton symport'
            else:
                ttype = 'Proton antiport'

        elif 'na1_p' in mets:
            s_p = r.metabolites[model.metabolites.get_by_id('na1_p')]
            s_m = r.metabolites[model.metabolites.get_by_id('{0}_p'.format(key))]
            if s_p*s_m == 1:
                ttype = 'Sodium symport'
            else:
                ttype = 'Sodium antiport'
        elif len(set([x[:-2] for x in mets]))==2:
            # Other symport
            sym = list(set([x[:-2] for x in mets if x[:-2]!= key]))[0]
            c0 = list(r.compartments)[0]
            s = r.metabolites[model.metabolites.get_by_id('{0}_{1}'.format(key, c0))]
            s_sym = r.metabolites[model.metabolites.get_by_id('{0}_{1}'.format(sym, c0))]
            if s*s_sym < 0:
                ttype = '{0} antiporter'.format(sym)
            else:
                ttype = '{0} symporter'.format(sym)
        elif r.id =='NMNt7pp':
            ttype = 'Other active transport'
            
        if (r.lower_bound < 0) and (r.upper_bound > 0):
            direction = 'Import and Export'
        elif (r.lower_bound == 0) and (r.upper_bound == 0):
            direction = 'Blocked'
        else:
            if (r.lower_bound < 0) and (r.upper_bound == 0):
                factor = -1
            else:
                factor = 1
            if 'e' in r.compartments:
                s = r.metabolites[model.metabolites.get_by_id('{0}_e'.format(key))]
            elif 'p' in r.compartments:
                s = r.metabolites[model.metabolites.get_by_id('{0}_p'.format(key))]
            else:
                print("###")
                print(r.reaction)
            if factor*s == -1:
                direction = 'Import'
            else:
                direction = 'Export'


        if (ttype =='None') or (membrane == '??'):
            print(r.id, r.reaction, r.bounds, r.name)
            print(r.compartments, compartments)
            print(r_id, ttype, membrane, direction)
            print('\n')
        
        name = model.metabolites.get_by_id('{0}_{1}'.format(key, list(r.compartments)[0])).name
        genes = ', '.join([g.id for g in r.genes if g.id !='s0001'])
        all_data.append([key, name, membrane, direction, ttype, r_id, genes])
                

In [171]:
df = pd.DataFrame(all_data, columns = ['iML1515 metabolite ID', 'Metabolite name', 'Membrane', 'Direction', 'Transport type', 'iML1515 transport reaction ID', 'Genes'])

In [147]:
# # Read metabolite mapping
# mapping_df = pd.read_csv('../data/id_mapping.csv', index_col=0)
# # df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)


In [172]:
df.to_csv('e_coli_transporters.csv')